In [ ]:
#AUC significa Área Sob Curva ROC.
#Isso mede o quão bem o mapa de saliência de uma imagem prevê as fixações humanas na imagem.
#A curva ROC é criada varrendo os valores limite
#determinado pelo intervalo de valores do mapa de saliência em locais de fixação.
#A taxa de verdadeiro positivo (tp) corresponde à proporção dos valores do mapa de saliência acima do limite
#em locais de fixação para o número total de locais de fixação.
#A taxa de falsos positivos (fp) corresponde à proporção dos valores do mapa de saliência acima do limite
#em todos os outros locais para o número total de outros locais possíveis (pixels de imagem não fixados).
#AUC=0,5 é o nível de "chance".
#Um valor de 1 indica uma classificação perfeita.


#A métrica AUC-Borji propôs amostrar aleatoriamente negativos de todos os locais de não fixação,
#que podem ser considerados como subconjunto da AUC-Judd.


import glob
import os
import matplotlib.pyplot as plt
#import numpy as np
#from PIL import Image
#import numpy

q_all = []
p_all = []
out_all = []

print ('Trabalhando nisso...')

def normalize(x, method='standard', axis=None):
	x = np.array(x, copy=False)
	if axis is not None:
		y = np.rollaxis(x, axis).reshape([x.shape[axis], -1])
		shape = np.ones(len(x.shape))
		shape[axis] = x.shape[axis]
		if method == 'standard':
			res = (x - np.mean(y, axis=1).reshape(shape)) / np.std(y, axis=1).reshape(shape)
		elif method == 'range':
			res = (x - np.min(y, axis=1).reshape(shape)) / (np.max(y, axis=1) - np.min(y, axis=1)).reshape(shape)
		elif method == 'sum':
			res = x / np.float_(np.sum(y, axis=1).reshape(shape))
		else:
			raise ValueError('method not in {"standard", "range", "sum"}')
	else:
		if method == 'standard':
			res = (x - np.mean(x)) / np.std(x)
		elif method == 'range':
			res = (x - np.min(x)) / (np.max(x) - np.min(x))
		elif method == 'sum':
			res = x / float(np.sum(x))
		else:
			raise ValueError('method not in {"standard", "range", "sum"}')
	return res

def AUC_Borji(saliency_map, fixation_map, n_rep=100, step_size=0.1, rand_sampler=None):
    
    # saliency_map is the saliency map
    # fixation_map is the human fixation map (binary matrix)
    
	#Parâmetros
	#----------
	#saliency_map : matriz de valor real
	#fixação_map : matriz binária
	#	Mapa de fixação humana.
	#jitter : booleano, opcional
	#	Se True (padrão), um pequeno número aleatório seria adicionado a cada pixel do mapa de saliência.
	#	Mapas de saliência de jitter que vêm de modelos de saliência que têm muitos valores zero.
	#	Se o mapa de saliência for feito com uma gaussiana, ele não precisa ser alterado
	#	pois os valores variam e não há um grande patch do mesmo valor.
	#	Na verdade, o jittering quebra a ordenação nos pequenos valores!
    #Resultados:
	#-------
	#AUC : float, entre [0,1]
                      
    
	saliency_map = np.array(saliency_map, copy=False)
	fixation_map = np.array(fixation_map, copy=False) > 0.5
	# If there are no fixation to predict, return NaN
	if not np.any(fixation_map):
		print('no fixation to predict')
		return np.nan
	# Make the saliency_map the size of the fixation_map
	if saliency_map.shape != fixation_map.shape:
		saliency_map = resize(saliency_map, fixation_map.shape, order=3, mode='constant')
	# Normalize saliency map to have values between [0,1]
	saliency_map = normalize(saliency_map, method='range')

	S = saliency_map.ravel()
	F = fixation_map.ravel()
	S_fix = S[F] # Saliency map values at fixation locations
	n_fix = len(S_fix)
	n_pixels = len(S)
	# For each fixation, sample n_rep values from anywhere on the saliency map
	if rand_sampler is None:
		#r = random.randint(0, n_pixels, [n_fix, n_rep])
		r = numpy.random.randint(0, n_pixels, [n_fix, n_rep])
		S_rand = S[r] # Saliency map values at random locations (including fixated locations!? underestimated)
	else:
		S_rand = rand_sampler(S, F, n_rep, n_fix)
        
	# Calculate AUC per random split (set of random locations)
	auc = np.zeros(n_rep) * np.nan
	for rep in range(n_rep):
		#thresholds = np.r_[0:np.max(np.r_[S_fix, S_rand[:,rep]]):step_size][::-1]
        
		thresholds = np.r_[S_fix, S_rand[:,rep]]   
		thresholds = np.max(thresholds)
		#print(thresholds)
		thresholds = np.r_[0:thresholds:step_size][::-1]       
        
		tp = np.zeros(len(thresholds)+2)
		fp = np.zeros(len(thresholds)+2)
		tp[0] = 0; tp[-1] = 1
		fp[0] = 0; fp[-1] = 1
		for k, thresh in enumerate(thresholds):
			tp[k+1] = np.sum(S_fix >= thresh) / float(n_fix)
			fp[k+1] = np.sum(S_rand[:,rep] >= thresh) / float(n_fix)
		auc[rep] = np.trapz(tp, fp)
	return np.mean(auc) # Average across random splits


if __name__ == '__main__':
    folder = r'...\saliency_all' #Diretório que contenha os mapas de saliência e os grounds truth

    gt_saliency_folder = folder + '\salmap' #mapas de saliencia do modelo
    gt_fixation_folder = folder + '\\fixmap' #mapas de saliencia - ground truth
    #print(gt_saliency_folder)
     
            
    for filename_sal in glob.glob(os.path.join(gt_saliency_folder,'*.png')):
        #print(filename_sal)
        p= plt.imread(filename_sal)
        p_all.append(p)
    for filename_fix in glob.glob(os.path.join(gt_fixation_folder,'*.jpeg')):
        #print(filename_fix)
        q= plt.imread(filename_fix)
        q_all.append(p)              
            
    for i in range(0,len(p_all)):
        out = AUC_Borji(p_all[i],p_all[i])
        out_all.append(out)
    soma = sum(out_all)
    media = soma/len(p_all)
    print (media) 